In [ ]:
# Install required packages for computer vision demo
# Snowflake Container Runtime supports pip install for open-source packages
%pip install torch torchvision transformers pillow requests matplotlib plotly


In [ ]:
# Import libraries and verify GPU availability
import torch
import torchvision.transforms as transforms
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from snowflake.snowpark.context import get_active_session
import pandas as pd
import requests
import io
import time

print("🚀 Environment Setup")
print("=" * 40)

# Verify GPU availability  
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"🔥 GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    device = torch.device("cuda")
    print("✅ Running on GPU - Optimal performance!")
else:
    device = torch.device("cpu")
    print("⚠️  Running on CPU - Consider switching to GPU compute pool")

# Initialize Snowflake session
session = get_active_session()
print(f"❄️  Connected to Snowflake: {session.get_current_account()}")
print("✅ Environment ready for computer vision demo!")


In [ ]:
# Load pre-trained computer vision model
print("🤖 Loading Computer Vision Model")
print("=" * 40)

start_time = time.time()

# Use Microsoft ResNet-50 for image classification
model_name = "microsoft/resnet-50"
print(f"📥 Downloading model: {model_name}")

# Load processor and model
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name)

# Move model to GPU if available  
model = model.to(device)
model.eval()

load_time = time.time() - start_time
print(f"✅ Model loaded in {load_time:.2f} seconds")
print(f"📊 Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"🔥 Running on: {device}")
print("🎯 Ready for satellite image classification!")


In [ ]:
# Query NGA imagery metadata from Snowflake
print("📡 Connecting to NGA Imagery Database")
print("=" * 40)

# Query high-quality satellite images for analysis
imagery_query = """
SELECT 
    image_id,
    s3_key,
    latitude,
    longitude,
    sensor_category,
    combined_quality_score,
    bay_region,
    closest_landmark,
    capture_date,
    file_size_bytes
FROM silver_imagery_metadata_iceberg 
WHERE s3_key IS NOT NULL 
    AND combined_quality_score >= 85
ORDER BY combined_quality_score DESC
LIMIT 10
"""

try:
    imagery_df = session.sql(imagery_query).to_pandas()
    print(f"🛰️  Found {len(imagery_df)} high-quality satellite images")
    print(f"📍 Locations: {', '.join(imagery_df['BAY_REGION'].unique())}")
    print(f"⭐ Average quality: {imagery_df['COMBINED_QUALITY_SCORE'].mean():.1f}%")
    
    # Display sample data
    print("\n📊 Sample Imagery Metadata:")
    display(imagery_df[['IMAGE_ID', 'BAY_REGION', 'SENSOR_CATEGORY', 'COMBINED_QUALITY_SCORE']].head())
    
except Exception as e:
    print(f"❌ Error loading data: {e}")
    print("💡 Ensure you have access to silver_imagery_metadata_iceberg table")
    # Create sample data for demo purposes
    imagery_df = pd.DataFrame({
        'IMAGE_ID': ['demo_image_1', 'demo_image_2'],
        'S3_KEY': ['sample1.jpg', 'sample2.jpg'],
        'BAY_REGION': ['San Francisco', 'Oakland'],
        'COMBINED_QUALITY_SCORE': [92.5, 89.1]
    })
    print("🔄 Using sample data for demonstration")


In [ ]:
# Computer Vision Analysis Function
def analyze_satellite_image(s3_key, session, model, processor, device):
    """
    GPU-accelerated computer vision analysis of satellite imagery
    """
    try:
        # Get presigned URL for secure S3 access
        url_query = f"SELECT GET_PRESIGNED_URL('@s3_imagery_stage_direct', '{s3_key}', 3600) as url"
        url_result = session.sql(url_query).collect()
        
        if not url_result:
            return None, "Could not generate presigned URL"
            
        image_url = url_result[0]['URL']
        
        # Download and process image
        response = requests.get(image_url, timeout=30)
        image = Image.open(io.BytesIO(response.content)).convert('RGB')
        
        # Preprocess image for model
        inputs = processor(image, return_tensors="pt").to(device)
        
        # GPU-accelerated inference
        start_time = time.time()
        with torch.no_grad():
            outputs = model(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        
        inference_time = time.time() - start_time
        
        # Get top 5 predictions
        top_predictions = torch.topk(predictions, 5)
        
        results = {
            'image': image,
            'predictions': [],
            'inference_time': inference_time,
            'device': str(device)
        }
        
        # Convert to readable format
        for score, idx in zip(top_predictions.values[0], top_predictions.indices[0]):
            label = model.config.id2label[idx.item()]
            confidence = score.item() * 100
            results['predictions'].append({
                'label': label,
                'confidence': confidence
            })
        
        return results, None
        
    except Exception as e:
        return None, f"Analysis error: {str(e)}"

print("🔬 Computer vision analysis function ready!")
print("⚡ Supports GPU-accelerated inference with secure S3 access")


In [ ]:
# GPU-Powered Computer Vision Demo
print("🚀 Starting GPU-Powered Satellite Image Analysis")
print("=" * 50)

analysis_results = []
total_start_time = time.time()

# Analyze first 3 images for demo
for idx, row in imagery_df.head(3).iterrows():
    print(f"\n📡 Analyzing Image {idx+1}")
    print(f"  ID: {row['IMAGE_ID'][:30]}...")
    if 'BAY_REGION' in row:
        print(f"  Location: {row['BAY_REGION']}")
    if 'COMBINED_QUALITY_SCORE' in row:
        print(f"  Quality: {row['COMBINED_QUALITY_SCORE']:.1f}%")
    
    # Perform analysis
    results, error = analyze_satellite_image(
        row['S3_KEY'], session, model, processor, device
    )
    
    if results:
        print(f"  ⚡ Inference: {results['inference_time']*1000:.1f}ms")
        print(f"  🔥 Device: {results['device']}")
        print(f"  🎯 Top prediction: {results['predictions'][0]['label'][:30]}")
        print(f"  📊 Confidence: {results['predictions'][0]['confidence']:.1f}%")
        
        # Display visualization
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # Show satellite image
        ax1.imshow(results['image'])
        ax1.set_title(f"Satellite Image: {row.get('BAY_REGION', 'Unknown Location')}")
        ax1.axis('off')
        
        # Show prediction confidence
        labels = [p['label'][:25] + ('...' if len(p['label']) > 25 else '') for p in results['predictions']]
        confidences = [p['confidence'] for p in results['predictions']]
        
        bars = ax2.barh(labels[::-1], confidences[::-1], color='steelblue')
        ax2.set_xlabel('Confidence (%)')
        ax2.set_title('Computer Vision Predictions')
        ax2.set_xlim(0, 100)
        
        # Add confidence labels
        for bar, conf in zip(bars, confidences[::-1]):
            ax2.text(conf + 1, bar.get_y() + bar.get_height()/2, 
                    f'{conf:.1f}%', va='center', fontsize=9)
        
        plt.tight_layout()
        plt.show()
        
        # Store results
        analysis_results.append({
            'image_id': row['IMAGE_ID'],
            'location': row.get('BAY_REGION', 'Unknown'),
            'top_prediction': results['predictions'][0]['label'],
            'confidence': results['predictions'][0]['confidence'],
            'inference_time_ms': results['inference_time'] * 1000,
            'device': results['device']
        })
        
    else:
        print(f"  ❌ Error: {error}")

total_time = time.time() - total_start_time
print(f"\n🎯 Analysis Complete!")
print(f"⏱️  Total time: {total_time:.2f} seconds")

if analysis_results:
    avg_inference = np.mean([r['inference_time_ms'] for r in analysis_results])
    print(f"⚡ Average inference: {avg_inference:.1f}ms")
    print(f"🚀 Throughput: {1000/avg_inference:.1f} images/second")
    print(f"🎯 Average confidence: {np.mean([r['confidence'] for r in analysis_results]):.1f}%")


In [ ]:
# Demo Summary & Performance Analytics
print("🎯 NGA Computer Vision Demo Summary")
print("=" * 50)

# Platform information
print(f"🚀 Platform: Snowflake Container Runtime")
print(f"🤖 Model: {model_name}")
print(f"📊 Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"⚡ Device: {device} ({'GPU-Accelerated' if torch.cuda.is_available() else 'CPU'})")

if analysis_results:
    # Convert to DataFrame for analysis
    df_results = pd.DataFrame(analysis_results)
    
    print(f"\n📈 Performance Metrics:")
    print(f"  Images Analyzed: {len(analysis_results)}")
    print(f"  Average Inference: {df_results['inference_time_ms'].mean():.1f}ms")
    print(f"  Throughput: {1000/df_results['inference_time_ms'].mean():.1f} images/sec")
    print(f"  Average Confidence: {df_results['confidence'].mean():.1f}%")
    
    # Interactive performance visualization
    if len(df_results) > 1:
        fig = go.Figure()
        
        # Performance chart
        fig.add_trace(go.Bar(
            x=df_results['location'],
            y=df_results['inference_time_ms'],
            name='Inference Time (ms)',
            marker_color='steelblue',
            text=[f"{x:.1f}ms" for x in df_results['inference_time_ms']],
            textposition='auto'
        ))
        
        fig.update_layout(
            title="🛰️ GPU Inference Performance by Location",
            xaxis_title="Location",
            yaxis_title="Inference Time (ms)",
            template="plotly_white",
            height=400
        )
        
        fig.show()
    
    # Results table
    print(f"\n📊 Detailed Results:")
    display(df_results[['location', 'top_prediction', 'confidence', 'inference_time_ms', 'device']])

print(f"\n🏛️  NGA Intelligence Capabilities Demonstrated:")
print("✅ GPU-accelerated computer vision on satellite imagery")
print("✅ Open-source ML libraries (PyTorch, Transformers)")
print("✅ Real-time inference on high-resolution data")
print("✅ Seamless Snowflake data platform integration")
print("✅ Production-ready container runtime infrastructure")

print(f"\n🚀 Next Steps for Production:")
print("• Scale to thousands of satellite images")
print("• Deploy custom models for specific intelligence tasks")
print("• Implement real-time streaming inference pipelines")
print("• Integrate with operational NGA intelligence workflows")

print(f"\n🎖️  Mission-Ready Intelligence Platform: OPERATIONAL ✅")
